In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '487/Project/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
os.chdir(GOOGLE_DRIVE_PATH)

Mounted at /content/drive
['label', 'train_images', 'model_weights.pt', 'image_embedding_VGG_finetuned.pt', 'memotion_train.csv', 'enhanced_train_images', 'compressed_train_images', 'model_weights_enhanced.pt', 'image_embedding_VGG_finetuned_enhanced.pt', 'model_weights_compressed.pt', 'val_images', 'Test Images', 'val_embedding_VGG_finetuned.pt', 'test_embedding_VGG_finetuned.pt', 'val_embedding_pretrained.pt', 'test_embedding_pretrained.pt', 'train_embedding_pretrained.pt', 'clip_embedding.pt']


In [2]:
!pip install torch torchvision
!pip install git+https://github.com/openai/CLIP.git

import torch
import clip
from PIL import Image
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-wemonrdc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-wemonrdc
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369398 sha256=a197ab5bf1d695c1860e7280abff714e4ff5c4dcfe1c9e582df3a83865b6a76d
  Stored in directory: /tmp/pip-ephem-wheel-cache-7uplwzoo/wheels/c8/e4/e1/11374c111387672fc2068dfbe0d4b424cb9cdd1b2e184a71b5
Successfully built clip


100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 293MiB/s]


In [3]:
df = pd.read_csv('memotion_val.csv')
strings = df['ocr_text'].astype(str).tolist()

In [ ]:
embed = ()
m_len = 75
for i in range(1500):
  if i % 100 == 0:
    print(i)
  # Load the image and preprocess it
  image = Image.open(f"val_images/{i+1}.jpg")
  image_input = preprocess(image).unsqueeze(0).to(device)

  # Encode the text input
  chunks = [strings[i][i:i+m_len] for i in range(0, len(strings[i]), m_len)]
  chunks = [clip.tokenize(chunk) for chunk in chunks]
  chunks = torch.stack(chunks)
  chunks = torch.mean(chunks.float(), dim=0)
  text = chunks.to(torch.int).to(device)

  # Get the embeddings
  with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text)
    # Concatenate the image and text features
    features = torch.cat((image_features, text_features), dim=1)

  embed = embed + (features,)

embed = torch.stack(embed)
torch.save(embed.squeeze(), 'clip_embedding_val.pt')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [ ]:
embed.shape

torch.Size([7000, 1, 1024])

In [ ]:
torch.save(embed.squeeze(), 'clip_embedding.pt')